In [4]:
import igraph as ig
import random

# 随机生成一个图
def generate_random_graph(num_vertices, num_edges):
    g = ig.Graph()
    g.add_vertices(num_vertices)
    
    # 随机添加边
    edges = set()
    while len(edges) < num_edges:
        v1, v2 = random.sample(range(num_vertices), 2)  # 随机选择两个不同的节点
        edges.add((v1, v2))
    
    g.add_edges(list(edges))
    return g

# 存储图为指定格式
def save_graph_to_file(graph, filename):
    with open(filename, 'w') as file:
        # 存储节点信息，假设节点标签为 'Node_<node_id>'
        for vertex in graph.vs:
            # file.write(f"v {vertex.index} Node_{vertex.index}\n")
            file.write(f"v {vertex.index}\n")
        
        # 存储边信息，假设边标签为 'Edge_<edge_id>'
        for edge in graph.es:
            v1, v2 = edge.source, edge.target
            # file.write(f"e {v1} {v2} Edge_{edge.index}\n")
            file.write(f"e {v1} {v2}\n")

# BFS遍历图
def bfs(graph, start_vertex):
    # 记录已访问的节点
    visited = set()
    queue = [start_vertex]
    visited.add(start_vertex)

    reachable_count = 0
    while queue:
        node = queue.pop(0)
        neighbors = graph.neighbors(node)
        
        for neighbor in neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                reachable_count += 1
                
    return reachable_count

# 示例使用
num_vertices = 100000  # 节点数
num_edges = 1500000  # 边数

# 随机生成图
graph = generate_random_graph(num_vertices, num_edges)

# 存储图到文件
filename = "random_graph.graph"
save_graph_to_file(graph, filename)
print(f"图已保存到文件: {filename}")

# 假设我们从节点1开始BFS
start_node = 1
reachable_neighbors = bfs(graph, start_node)

print(f"从节点 {start_node} 可以访问到 {reachable_neighbors} 个邻居节点。")


图已保存到文件: random_graph.graph
从节点 1 可以访问到 99999 个邻居节点。


In [45]:
import igraph as ig
import random
# import igraph as ig
import leidenalg
import matplotlib.pyplot as plt

# 随机生成一个图
def generate_random_graph(num_vertices, num_edges):
        # 创建一个简单的图
    # 示例图：10个节点，每个节点随机连接
    g = ig.Graph.Erdos_Renyi(n=num_vertices, m=num_edges)

    # 为每个节点指定名称
    g.vs["name"] = [str(i) for i in range(g.vcount())]

    return g

# 使用Leiden算法进行社区检测
def apply_leiden_algorithm(graph):

    partition = leidenalg.find_partition(graph, leidenalg.ModularityVertexPartition)
    # # 打印每个节点所在的社区
    # print("\n每个节点所在的社区：")
    # for community, i in enumerate(partition):
    #     print(f"节点 {i} 属于社区 {community}")
    return partition

# 存储图为指定格式
def save_graph_to_file2(graph, communities, filename):
    with open(filename, 'w') as file:
        # 按照社区顺序存储节点
        node_order = []
        for community_index, community in enumerate(communities):
            for node in community:
                node_order.append((node, community_index))  # 将节点和社区ID一起存储

        # 存储节点信息
        for vertex, community_id in node_order:
            file.write(f"v {vertex} {community_id}\n")  # 添加社区ID作为后缀

        # 存储边信息
        for edge in graph.es:
            v1, v2 = edge.source, edge.target
            community_v1 = next(community_id for node, community_id in node_order if node == v1)
            community_v2 = next(community_id for node, community_id in node_order if node == v2)
            file.write(f"e {v1} {community_v1} {v2} {community_v2}\n")



# 存储图为指定格式
def save_graph_to_file(graph, communities, filename):
    with open(filename, 'w') as file:
        # 按照社区顺序存储节点
        node_order = []
        for community_index, community in enumerate(communities):
            for node in community:
                node_order.append((node, community_index))  # 将节点和社区ID一起存储

        # 存储节点信息
        for vertex, community_id in node_order:
            file.write(f"v {vertex} {community_id}\n")  # 添加社区ID作为后缀

        # 先插入同一社区内的边，再插入跨社区的边
        # 创建一个字典来映射每个节点到其所属的社区
        node_to_community = {node: community_index for community_index, community in enumerate(communities) for node in community}

        # 插入同一社区内的边
        for edge in graph.es:
            v1, v2 = edge.source, edge.target
            community_v1 = node_to_community[v1]
            community_v2 = node_to_community[v2]
            # 如果 v1 和 v2 属于同一社区
            if community_v1 == community_v2:
                file.write(f"e {v1} {community_v1} {v2} {community_v2}\n")

        print(f"同一社区内的边已插入")

        # 插入跨社区的边
        for edge in graph.es:
            v1, v2 = edge.source, edge.target
            community_v1 = node_to_community[v1]
            community_v2 = node_to_community[v2]
            # 如果 v1 和 v2 不属于同一社区
            if community_v1 != community_v2:
                file.write(f"e {v1} {community_v1} {v2} {community_v2}\n")

        print(f"跨社区的边已插入")


# BFS遍历图
def bfs(graph, start_vertex):
    # 记录已访问的节点
    visited = set()
    queue = [start_vertex]
    visited.add(start_vertex)

    reachable_count = 0
    while queue:
        node = queue.pop(0)
        neighbors = graph.neighbors(node)
        
        for neighbor in neighbors:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                reachable_count += 1
                
    return reachable_count

# 示例使用
num_vertices = 100  # 节点数
num_edges = 1500  # 边数

# 随机生成图
graph = generate_random_graph(num_vertices, num_edges)

print(f"图已生成")

# 使用Leiden算法进行社区检测
communities = apply_leiden_algorithm(graph)

print(f"图已按Leiden社区划分")
print(f"社区数量：{len(communities)}")
print(f"每个社区的节点数量：{[len(community) for community in communities]}")

# 存储图到文件（按照社区重排）
filename = "random_graph_leiden_02.graph"
save_graph_to_file(graph, communities, filename)

correct_graph = "random_graph_correct_02.graph"
save_graph_to_file2(graph, communities, correct_graph)

# print(f"图已按Leiden社区划分并保存到文件: {filename}")

# 假设我们从节点1开始BFS
start_node = 1
reachable_neighbors = bfs(graph, start_node)

print(f"从节点 {start_node} 可以访问到 {reachable_neighbors} 个邻居节点。")


图已生成
图已按Leiden社区划分
社区数量：6
每个社区的节点数量：[21, 20, 20, 16, 14, 9]
同一社区内的边已插入
跨社区的边已插入
从节点 1 可以访问到 99 个邻居节点。
